# Exercícios - Fontes de Dados - Parte 2 💾 

## Web Scraping da lista das maiores cidades do Wikipedia

O objetivo deste exercício é extrair da Wikipedia a lista das maiores cidades do mundo e sua população estimada, e organizar os dados para análises posteriores.

A Wikipedia tem uma página intitulada "Lista das cidades mais populosas do mundo", que provê uma lista das cidades mais populosas do mundo e o tamanho da sua população. A sua tarefa é escrever um *script* para "raspar" esta página e extrair as seguintes informações:
1. Nome da cidade
2. População estimada em 2018 (de acordo com dados da ONU)
3. País onde se situa a cidade

Após raspar os dados, crie um arquivo do tipo .csv com o seguinte cabeçalho:

```
Cidade,População,País
```

Armazene os dados raspados neste arquivo.

*Dicas*: Inspecione a página da Wikipedia antes usando as ferramentas do desenvolvedor em seu *web browser* para entender a estrutura da tabela e as tags que contêm os dados de que você necessita.

Recursos:

Lista das cidades mais populosas do mundo: https://pt.wikipedia.org/wiki/Lista_das_cidades_mais_populosas_do_mundo

Documentação da biblioteca BeautifulSoup: https://pypi.org/project/beautifulsoup4/

In [10]:
import requests
from bs4 import BeautifulSoup
import csv

def raspar_wikipedia_maiores_cidades():
    url = "https://pt.wikipedia.org/wiki/Lista_das_cidades_mais_populosas_do_mundo"
    response = requests.get(url)
    response.raise_for_status()  # Levantar exceção para quaisquer erros HTTP.

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Localizando a tabela com os dados da cidade
    tabela = soup.find('table', {'class': 'wikitable'})
    
    # Extracting data from the table
    dados_cidades = []
    rows = tabela.find_all('tr')[1:]  # Skipping the header row
    
    for row in rows:
        columns = row.find_all('td')
        
        if len(columns) >= 3:  # Garanta que há colunas suficientes para a extração de dados.
            cidade = columns[0].get_text(strip=True)
            populacao = columns[2].get_text(strip=True)
            pais = columns[1].get_text(strip=True)
            
            dados_cidades.append([cidade, populacao, pais])
    
    # Escrevendo os dados no arquivo csv.
    with open('./data/maiores_cidades.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['cidade', 'populacao', 'pais'])
        writer.writerows(dados_cidades)

    

In [11]:
raspar_wikipedia_maiores_cidades()

## 🌶️ Desafio! De 👀 nas despesas do Governo Federal

Já vimos, na aula passada, como acessar os dados de despesas dos deputados federais através do portal **Dados Abertos** da Câmara dos Deputados. Nós vamos agora prepara um *dataset* para analisar as despesas por função do Governo Federal. Apesar dos dados estarem em um portal de dados abertos, que também provê APIs para acessá-los, vamos utilizar também a técnica de *webscraping* para acessar e baixar os datasets para que possamos exercitar bem esse método.

O objetivo deste exercício é extrair do portal Tesouro Transparente as bases de despesas por Função do Governo Central, segundo a classificação COFOG (não se preocupe por enquanto com essa classificação). Você deverá buscar e baixar as últimas 3 bases disponíveis para análises posteriores. A função para baixar o arquivo em sua máquina local já foi dada.

⚠️ Atenção! Baixe somente os arquivos do tipo .csv.

*Dicas*: 
1. Inspecione a página dos datasets antes usando as ferramentas do desenvolvedor em seu *web browser* para entender a estrutura da página e as tags que contêm os arquivos de que você necessita.

2. Insira um timer de 60s depois do download de cada arquivo para evitar bloqueio do servidor (vai que ele acha que você o está atacando... 🧐)

Recursos:

Url das bases de dados: https://www.tesourotransparente.gov.br/ckan/dataset/despesas-por-funcao-do-governo-central-classificacao-cofog

In [1]:
import os

def baixar_arquivo(url, local_filename=None):
    """Baixa e salva o arquivo de uma url."""
    # Se o nome do arquivo não foi passado como parâmetro, extrair da url.
    if local_filename is None:
        local_filename = os.path.join("./data", url.split('/')[-1])

    #verifica se arquivo já foi baixado
    if os.path.exists(local_filename):
        print(f'Arquivo {local_filename} já baixado!')
        return None
    else:
        print('Solicitando arquivo do servidor.')
        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(local_filename, 'wb') as f:
            print(f"Baixando arquivo da url: {url}")
            for chunk in response.iter_content(chunk_size=8192): 
                f.write(chunk)
            return "Arquivo baixado!"

def eh_arquivo_csv(link):
    """Verifica se arquivo é do tipo .csv."""
    term_link = link[len(link)-3:]
    if term_link == 'csv':
        return True
    return False

In [2]:
def read_page(url):
    print(f'Fazendo requisição GET para o link: {url}')
    response = requests.get(url)
    response.raise_for_status() #Levanta uma exceção caso haja erro na requisição.
    
    if response.status_code == 200:
        content = response.text
        return content

In [3]:
import requests
from bs4 import BeautifulSoup
import time

TIME_TO_SLEEP = 120

url = 'https://www.tesourotransparente.gov.br/ckan/dataset/despesas-por-funcao-do-governo-central-classificacao-cofog'

content = read_page(url)
soup = BeautifulSoup(content, 'html.parser')

Fazendo requisição GET para o link: https://www.tesourotransparente.gov.br/ckan/dataset/despesas-por-funcao-do-governo-central-classificacao-cofog


In [4]:
# Primeiro, busca a tag <section> com id 'dataset-resources'
section = soup.find('section', id='dataset-resources')

# Se encontrada, então busque os elementos <ul> dentro dela.
if section:
    resource_list = section.find('ul', class_='resource-list')
    
    # If the <ul> is found, target <li> tags with class 'resource-item' inside it
    if resource_list:
        resource_items = resource_list.find_all('li', class_='resource-item')

        #Vamos pegar os 3 primeiros arquivos .csv. Sempre verifique a estrutura da página web
        #para evitar erros!
        for i in range(1,4):
            item = resource_items[i]
            
            # Busca a tag <a> dentro da tag <div> de id = 'dropdown btn-group'
            div_class = item.find('div', class_="dropdown btn-group")
            
            #busca a referência de download do arquivo
            if div_class:
                download_link = div_class.find('a', class_='resource-url-analytics')
                if download_link:
                    #Vamos baixar somentes os arquivos cs
                    link = download_link['href']
                    print(link)
                    
                    if eh_arquivo_csv(link):
                        if baixar_arquivo(link):
                            print('Esperando 120s para solicitar próximo download...')
                            time.sleep(TIME_TO_SLEEP)

                else:
                    print("Link para download não encontrado.")

https://www.tesourotransparente.gov.br/ckan/dataset/22d13d17-bf69-4a1a-add2-25cc1e25f2d7/resource/db8d7b1d-b581-4bc9-a9aa-447f0d29f301/download/Base-COFOG-2022.csv
Solicitando arquivo do servidor.
Baixando arquivo da url: https://www.tesourotransparente.gov.br/ckan/dataset/22d13d17-bf69-4a1a-add2-25cc1e25f2d7/resource/db8d7b1d-b581-4bc9-a9aa-447f0d29f301/download/Base-COFOG-2022.csv
Esperando 120s para solicitar próximo download...
https://www.tesourotransparente.gov.br/ckan/dataset/22d13d17-bf69-4a1a-add2-25cc1e25f2d7/resource/83b16dfe-9fbb-44ea-8b89-1f8d1b5d78e3/download/Base-COFOG-2021.csv
Solicitando arquivo do servidor.
Baixando arquivo da url: https://www.tesourotransparente.gov.br/ckan/dataset/22d13d17-bf69-4a1a-add2-25cc1e25f2d7/resource/83b16dfe-9fbb-44ea-8b89-1f8d1b5d78e3/download/Base-COFOG-2021.csv
Esperando 120s para solicitar próximo download...
https://www.tesourotransparente.gov.br/ckan/dataset/22d13d17-bf69-4a1a-add2-25cc1e25f2d7/resource/b258f856-b80e-4cad-a4bb-dc570b